In [ ]:
from __future__ import print_function
from keras.models import load_model
import numpy as np
import random
from src.read_data import read_trump_speeches
from src.utils import print_tensorflow_devices
from src.utils import sample

In [ ]:
# Parameters
seq_len = 21 # includes next word.
batch_size = 128
epochs = 50

In [ ]:
speeches = read_trump_speeches('data/speeches.txt')
words = np.unique(speeches)
word_index = dict((c, i) for i, c in enumerate(words))
index_word = dict((i, c) for i, c in enumerate(words))
n_words = len(words)

# Index the speeches, then create sentences of length 'seq_len' that we can train the RNN on.
speeches_indexed = [word_index[x] for x in speeches]
sentence_ranges = [range(i, i + seq_len) for i in range(0, len(speeches) - seq_len)]
sentences = [[speeches[y] for y in x] for x in sentence_ranges]
sentences_indexed = [[speeches_indexed[y] for y in x] for x in sentence_ranges]

# Train-test split
random.seed(12345)
random.shuffle(sentences_indexed)
train_split = int(0.95 * len(sentences_indexed))
sentences_indexed_train = sentences_indexed[:train_split]
sentences_indexed_test = sentences_indexed[train_split:]
print('Train: ' + str(len(sentences_indexed_train)))
print('Test: ' + str(len(sentences_indexed_test)))


In [ ]:
prediction_model = '20180902-040431.h5'
model = load_model('models/' + prediction_model)

examples_file = open('examples/' + prediction_model + '.txt', 'w')

for i in range(10):
    # Randomly pick a seed sequence
    seed = (sentences_indexed)[np.random.randint(len(sentences_indexed))]
    sentence = seed.copy()
    examples_file.write( '----- Generating with seed: "' + ' '.join([index_word[x] for x in seed]) + '"\n\n')

    for diversity in [0.3, 0.4, 0.5, 0.6, 0.7]:
        
        examples_file.write('Diversity: ' + str(diversity) + '\n')
        sentence = seed
        full_sentence = sentence.copy()

        for i in range(250):
            sentence = sentence[1:]
            x_pred = np.zeros((1, seq_len - 1, n_words), dtype=np.bool)
            for t, w in enumerate(sentence):
                x_pred[0, t, w] = 1

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            sentence.append(next_index)
            full_sentence.append(next_index)

        full_sentence = [index_word[x] for x in full_sentence]
        full_sentence = ' '.join(full_sentence)
        examples_file.write(full_sentence)
        examples_file.write('\n\n')
    examples_file.write('\n\n')

examples_file.close()